In [1]:
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def get_data():
    movie_data = pd.read_excel("data_places.xlsx")
    movie_data['name'] = movie_data['name'].str.lower()
    return movie_data

In [3]:
get_data()

,name,desc,address,image
0,shaniwar wada palace,A prominent historical landmark in Pune is Sha...,"Shaniwar Peth, Pune, Maharashtra 411030",https://images.thrillophilia.com/image/upload/...
1,aga khan palace,Yet another amazing landmark and one of the fa...,"Pune Nagar Road, Kalyani Nagar, Pune, Maharash...",https://images.thrillophilia.com/image/upload/...
2,dagdusheth halwai temple,A famous Ganesha temple in Pune is Shreemath D...,"Ganpati Bhavan, 250, Budhvar Peth, Shivaji Roa...",https://images.thrillophilia.com/image/upload/...
3,osho ashram,Osho Ashram is one of the famous tourist place...,"17, Koregaon Park Road, Vasani Nagar, Koregaon...",https://images.thrillophilia.com/image/upload/...
4,vetal tekdi,A prominent hill in the city limits of Pune is...,"Pashan, Pune 411038, Maharashtra",https://images.thrillophilia.com/image/upload/...
5,lal mahal,Lal Mahal is one of the famous tourist places ...,"Lal Mahal, Ganesh Rd, Durvankur Society, Kasba...",https://images.thrillophilia.com/image/upload/...
6,raja dinkar kelkar museum,An intriguing museum with a lot of references ...,"1377-78, Natu Baug,Off. Bajirao Road,Shukrawar...",https://images.thrillophilia.com/image/upload/...
7,national defence academy,NDA is the sophisticated acronym of the Nation...,"NDA Alumni Association, National Defence Acade...",https://images.thrillophilia.com/image/upload/...
8,explore parvati hill,This beautiful lush green hill is one of the p...,"Parvati Hill, Parvati Paytha, Pune, Maharashtr...",https://images.thrillophilia.com/image/upload/...
9,sinhagad fort,Sinhagad or Kondana is a pride of the southwe...,Sinhagad fort is located at Sinhagad Ghat Road...,https://res.cloudinary.com/thrillophilia/image...


In [4]:
def combine_data(data):
    data_recommend = data
    data_recommend['combine'] = data_recommend[data_recommend.columns[0:4]].apply(
                                                                         lambda x: ','.join(x.dropna().astype(str)),axis=1)
    data_recommend = data.drop(columns = ['name','address','desc','image'])
    return data_recommend

In [5]:
def transform_data(data_combine, data_plot):
        count = CountVectorizer(stop_words='english')
        count_matrix = count.fit_transform(data_combine['combine'])

        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(data_plot['desc'])

        combine_sparse = sp.hstack([count_matrix, tfidf_matrix], format='csr')
        cosine_sim = cosine_similarity(combine_sparse, combine_sparse)
        
        return cosine_sim


In [6]:
def recommend_movies(title, data, combine, transform):
        indices = pd.Series(data.index, index = data['name'])
        index = indices[title]
        
        sim_scores = list(enumerate(transform[index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:21]


        places_indices = [i[0] for i in sim_scores]

        places_title = data['name'].iloc[places_indices]
        places_desc = data['desc'].iloc[places_indices]
        places_add = data['address'].iloc[places_indices]
        places_image = data['image'].iloc[places_indices]

        recommendation_data = pd.DataFrame(columns=['Name', 'Description', 'Address', 'Image'])

        recommendation_data['Name'] = places_title
        recommendation_data['Description'] = places_desc
        recommendation_data['Address'] = places_add
        recommendation_data['Image'] = places_image

        return recommendation_data

In [7]:
def results(place_name):
        place_name = place_name.lower()

        find_place = get_data()
        combine_result = combine_data(find_place)
        transform_result = transform_data(combine_result,find_place)

        if place_name not in find_place['name'].unique():
                return 'Place not in Database'

        else:
                recommendations = recommend_movies(place_name, find_place, combine_result, transform_result)
                return recommendations.to_dict('records')

In [9]:
results('panshet waterpark')

[{'Name': 'diamond water park',
  'Description': 'Enthrall yourself with an eventful day at Diamond Water Park the next time you are visiting Pune. It is one of the main attractions in Pune where you can be assured of a wonderful day and jam-packed enjoyment for kids. Diamond Water Park is visited by the people for a refreshing retreat with water sports and other fun activities. It boasts of having more than 7 swimming pools and more than 32 international water rides and attractions. Some popular attractions at Diamond Water Park are kiddies pool, family play station, wet bubble, Tortuga Falls etc. the fun rides offer a thrilling experience to all the tourists and there is also an unlimited lunch which you can enjoy at Diamond Water Park.',
  'Address': 'Water Park Road, Lohegaon, Pune City, Maharashtra',
  'Image': 'https://res.cloudinary.com/thrillophilia/image/upload/c_fill,dpr_1.0,f_auto,fl_progressive.strip_profile,g_center,h_450,q_auto,w_753/v1/filestore/rovrjzytr433rdbuzumk2aogx